In [1]:
from collections import defaultdict
import torch
import numpy as np

from collections import defaultdict

def read_graph(file_path):
    """
    Reads a weighted directed graph from a file. Each line contains three values:
    start vertex, end vertex, and edge weight.

    Args:
        file_path (str): Path to the file containing the graph.

    Returns:
        edges (list): List of tuples representing directed edges (start, end, weight, edge_id).
    """
    edges = []
    edge_id = 0  # Unique identifier for each edge
    try:
        with open(file_path, 'r') as file:
            for line in file:
                line = line.strip()
                if line:  # Skip empty lines
                    start, end, weight = map(float, line.split())
                    edges.append((int(start), int(end), weight, edge_id))
                    edge_id += 1
    except Exception as e:
        print(f"Error reading graph: {e}")
    return edges


def initialize_graph(edges):
    """
    Converts edge list to an adjacency list and weights dictionary.
    Handles parallel edges by including edge_id as part of the structure.

    Args:
        edges (list): List of edges as (start, end, weight, edge_id).

    Returns:
        graph (dict): Adjacency list with (neighbor, edge_id) pairs.
        weights (dict): Dictionary mapping (start, end, edge_id) to weights.
    """
    graph = defaultdict(list)
    weights = {}
    for u, v, w, edge_id in edges:
        graph[u].append((v, edge_id))
        weights[(u, v, edge_id)] = w
    return graph, weights


def find_cycles_and_reduce(graph, weights, n):
    """
    Phase 1: Find cycles and reduce weights using a copy.
    Handles graphs with parallel edges by considering edge identifiers.

    Args:
        graph (dict): Adjacency list with (neighbor, edge_id) pairs.
        weights (dict): Dictionary mapping (u, v, edge_id) to weights.
        n (int): Number of vertices in the graph.

    Returns:
        removed_edges (set): Set of removed edges as (u, v, edge_id).
        removed_weights (dict): Dictionary of removed edges with their original weights.
    """
    weights_copy = weights.copy()  # Work with a copy of weights
    removed_edges = set()
    removed_weights = {}

    while True:
        cycle = find_cycle(graph, n)  # Modified `find_cycle` returns edges with edge_id
        if not cycle:  # No cycle found
            break

        # Ensure all edges in the cycle exist in the weights dictionary
        cycle = [(u, v, edge_id) for u, v, edge_id in cycle if (u, v, edge_id) in weights_copy]

        if not cycle:  # If no valid cycle exists, continue
            continue

        # Find the minimum weight in the cycle
        min_weight = min(weights_copy[(u, v, edge_id)] for u, v, edge_id in cycle)

        for u, v, edge_id in cycle:
            weights_copy[(u, v, edge_id)] -= min_weight
            if weights_copy[(u, v, edge_id)] <= 0:
                # Ensure the edge is in the graph before removing
                if (v, edge_id) in graph[u]:
                    graph[u].remove((v, edge_id))
                    removed_edges.add((u, v, edge_id))
                    removed_weights[(u, v, edge_id)] = weights[(u, v, edge_id)]

    return removed_edges, removed_weights

from collections import deque

def find_cycle(graph, n):
    """
    Detect a cycle in the graph using DFS and return the cycle as a list of edges.
    Handles parallel edges and cycles of length 2 caused by reverse edges.

    Args:
        graph (dict): Adjacency list with (neighbor, edge_id) pairs.
        n (int): Number of vertices in the graph.

    Returns:
        cycle (list): List of edges forming the cycle, or None if no cycle is found.
    """
    visited = [False] * n
    stack = [False] * n
    parent = [-1] * n
    edge_to_parent = {}  # Map to track edge_id for cycle reconstruction

    def dfs(v):
        visited[v] = True
        stack[v] = True
        for neighbor, edge_id in graph[v]:
            if not visited[neighbor]:
                parent[neighbor] = v
                edge_to_parent[neighbor] = edge_id
                cycle = dfs(neighbor)
                if cycle:
                    return cycle
            elif stack[neighbor]:
                # Found a cycle, reconstruct it
                cycle = []
                current = v
                while current != neighbor:
                    if current not in edge_to_parent:
                        break  # Avoid KeyError if edge metadata is missing
                    cycle.append((parent[current], current, edge_to_parent[current]))
                    current = parent[current]

                # Handle the root of the cycle
                if neighbor in edge_to_parent and parent[neighbor] != -1:
                    cycle.append((parent[neighbor], neighbor, edge_to_parent[neighbor]))
                return cycle

        stack[v] = False
        return None

    # Detect length-2 cycles caused by reverse edges
    for u in list(graph):  # Use list(graph) to iterate over a static copy of keys
        for neighbor, edge_id1 in graph[u]:
            for neighbor_of_neighbor, edge_id2 in graph[neighbor]:
                if neighbor_of_neighbor == u and edge_id1 != edge_id2:
                    # Found a length-2 cycle
                    return [(u, neighbor, edge_id1), (neighbor, u, edge_id2)]

    # Run DFS for longer cycles
    for i in range(n):
        if not visited[i]:
            cycle = dfs(i)
            if cycle:
                return cycle

    return None




from collections import deque

import numpy as np

def find_minimum_weight_cycle(graph, weights, n):
    """
    Find the minimum weight cycle in the graph using Floyd-Warshall.

    :param graph: Adjacency list representation of the graph.
    :param weights: Dictionary of edge weights.
    :param n: Total number of vertices in the graph.
    :return: List of edges representing the minimum weight cycle.
    """
    # Step 1: Initialize distance and predecessor matrices
    dist = np.full((n, n), float('inf'))
    pred = [[-1 for _ in range(n)] for _ in range(n)]

    # Fill in the distances based on edge weights
    for u in range(n):
        dist[u][u] = 0
        if u in graph:
            for v in graph[u]:
                dist[u][v] = weights.get((u, v), float('inf'))
                pred[u][v] = u

    # Step 2: Run Floyd-Warshall algorithm
    for k in range(n):
        for i in range(n):
            for j in range(n):
                if dist[i][j] > dist[i][k] + dist[k][j]:
                    dist[i][j] = dist[i][k] + dist[k][j]
                    pred[i][j] = pred[k][j]

    # Step 3: Find the minimum weight cycle
    min_cycle_weight = float('inf')
    cycle = []

    for u in range(n):
        for v in range(n):
            if u != v and dist[u][v] < float('inf') and dist[v][u] < float('inf'):
                cycle_weight = dist[u][v] + dist[v][u]
                if cycle_weight < min_cycle_weight:
                    min_cycle_weight = cycle_weight
                    # Reconstruct the cycle
                    cycle = []
                    # Trace path from u to v
                    current = v
                    while current != u:
                        cycle.append((pred[u][current], current))
                        current = pred[u][current]
                    # Trace path from v back to u
                    current = u
                    while current != v:
                        cycle.append((pred[v][current], current))
                        current = pred[v][current]

    # Return the minimum weight cycle if found
    if min_cycle_weight == float('inf'):
        return None
    else:
        return cycle


def check_and_readd_edges(graph, removed_edges, n):
    """
    Phase 2: Check and re-add edges if they do not create a cycle.
    Handles graphs with parallel edges using edge identifiers.

    Args:
        graph (dict): Adjacency list with (neighbor, edge_id) pairs.
        removed_edges (set): Set of removed edges as (u, v, edge_id).
        n (int): Number of vertices in the graph.

    Returns:
        readded_edges (set): Set of edges that were successfully re-added.
        remaining_removed_edges (set): Set of edges that could not be re-added.
    """
    
    def has_path(start, end, graph):
        """
        Helper function to check if there is a path from start to end using DFS.
        Avoids cycles when re-adding edges.
        """
        visited = [False] * n
        stack = [start]
        while stack:
            node = stack.pop()
            if node == end:
                return True
            if not visited[node]:
                visited[node] = True
                stack.extend(neighbor for neighbor, _ in graph[node])  # Add only neighbors
        return False

    readded_edges = set()
    removed_edges_list = sorted(list(removed_edges), reverse=True)

    for u, v, edge_id in removed_edges_list:
        if not has_path(v, u, graph):  # Only re-add if it doesn't create a cycle
            graph[u].append((v, edge_id))
            readded_edges.add((u, v, edge_id))

    remaining_removed_edges = removed_edges - readded_edges
    return readded_edges, remaining_removed_edges

def mwfas(file_path):
    """
    Main function to find Minimum Weighted Feedback Arc Set (MWFAS) in a graph with parallel edges.
    
    :param file_path: Path to the file containing the graph.
    :return: A dictionary with metrics, updated graph, removed edges, and their weights.
    """
    # Read the graph and initialize its structure
    edges = read_graph(file_path)
    n = max(max(u, v) for u, v, _, _ in edges) + 1
    graph, weights = initialize_graph(edges)

    # Original graph statistics
    total_edges = len(edges)
    total_weight = sum(w for _, _, w, _ in edges)

    # Phase 1: Reduce cycles
    removed_edges, removed_weights = find_cycles_and_reduce(graph, weights, n)

    # Phase 2: Re-add edges (if applicable)
    readded_edges, remaining_removed_edges = check_and_readd_edges(graph, removed_edges, n)

    # Compute final metrics
    num_removed_edges = len(remaining_removed_edges)
    total_removed_weight = sum(removed_weights.get(edge, 0) for edge in remaining_removed_edges)

    # Return results
    return {
        "total_edges": total_edges,
        "total_weight": total_weight,
        "num_removed_edges": num_removed_edges,
        "removed_weight": total_removed_weight,
        "final_graph": graph,
        "removed_edges": remaining_removed_edges,
        "removed_weights": {edge: removed_weights.get(edge, 0) for edge in remaining_removed_edges},
    }

    

# Example usage




In [2]:
import random
from collections import defaultdict, deque


class DirectedGraph:
    def __init__(self, vertices):
        self.graph = defaultdict(list)
        self.vertices = vertices

    def add_edge(self, src, dest, weight=1):
        self.graph[src].append((dest, weight))

    def get_edges(self):
        edges = []
        for src in self.graph:
            for dest, weight in self.graph[src]:
                edges.append((src, dest, weight))
        return edges

    def remove_edge(self, src, dest):
        self.graph[src] = [(d, w) for d, w in self.graph[src] if d != dest]

    def get_indegree(self):
        indegree = {v: 0 for v in range(self.vertices)}
        for src in self.graph:
            for dest, _ in self.graph[src]:
                indegree[dest] += 1
        return indegree

    def get_outdegree(self):
        outdegree = {v: 0 for v in range(self.vertices)}
        for src in self.graph:
            for dest, _ in self.graph[src]:
                outdegree[src] += 1
        return outdegree

    def eliminate_parallel_arcs(self):
        for src in self.graph:
            seen = {}
            for dest, weight in self.graph[src]:
                if dest in seen:
                    seen[dest] += weight
                else:
                    seen[dest] = weight
            self.graph[src] = [(dest, weight) for dest, weight in seen.items()]

    def eliminate_two_cycles(self):
        for src in list(self.graph.keys()):
            for dest, weight in self.graph[src]:
                for back_dest, back_weight in self.graph[dest]:
                    if back_dest == src:
                        if weight > back_weight:
                            self.graph[src] = [(d, w) for d, w in self.graph[src] if d != dest]
                            self.graph[dest] = [(d, w) for d, w in self.graph[dest] if d != src]
                        elif weight == back_weight:
                            self.graph[src] = [(d, w) for d, w in self.graph[src] if d != dest]
                            self.graph[dest] = [(d, w) for d, w in self.graph[dest] if d != src]

    def strongly_connected_components(self):
        index = 0
        stack = []
        indices = [-1] * self.vertices
        lowlinks = [-1] * self.vertices
        on_stack = [False] * self.vertices
        sccs = []

        def strongconnect(v):
            nonlocal index
            indices[v] = index
            lowlinks[v] = index
            index += 1
            stack.append(v)
            on_stack[v] = True

            for neighbor, _ in self.graph[v]:
                if indices[neighbor] == -1:
                    strongconnect(neighbor)
                    lowlinks[v] = min(lowlinks[v], lowlinks[neighbor])
                elif on_stack[neighbor]:
                    lowlinks[v] = min(lowlinks[v], indices[neighbor])

            if lowlinks[v] == indices[v]:
                scc = []
                while True:
                    node = stack.pop()
                    on_stack[node] = False
                    scc.append(node)
                    if node == v:
                        break
                sccs.append(scc)

        for v in range(self.vertices):
            if indices[v] == -1:
                strongconnect(v)

        return sccs

class HCSReduction:
    def __init__(self, graph):
        self.graph = graph

    def reduce(self):
        self.graph.eliminate_parallel_arcs()
        self.graph.eliminate_two_cycles()
        sccs = self.graph.strongly_connected_components()
        for scc in sccs:
            if len(scc) > 1:
                self.contract_scc(scc)

    def contract_scc(self, scc):
        super_node = scc[0]
        for node in scc[1:]:
            for neighbor, weight in self.graph.graph[node]:
                self.graph.add_edge(super_node, neighbor, weight)
            self.graph.graph.pop(node, None)

class GreedyABS:
    def __init__(self, graph):
        self.graph = graph

    def solve(self):
        indegree = self.graph.get_indegree()
        outdegree = self.graph.get_outdegree()

        sources = deque([v for v in indegree if indegree[v] == 0])
        sinks = deque([v for v in outdegree if outdegree[v] == 0])

        ordering = []

        while sources or sinks:
            while sources:
                v = sources.popleft()
                ordering.append(v)
                for neighbor, _ in self.graph.graph[v]:
                    indegree[neighbor] -= 1
                    if indegree[neighbor] == 0:
                        sources.append(neighbor)

            while sinks:
                v = sinks.popleft()
                ordering.append(v)
                for src in list(self.graph.graph.keys()):
                    self.graph.graph[src] = [(d, w) for d, w in self.graph.graph[src] if d != v]
                    if not self.graph.graph[src]:
                        sinks.append(src)

        if len(ordering) < self.graph.vertices:
            remaining = [v for v in range(self.graph.vertices) if v not in ordering]
            ordering.extend(remaining)

        return ordering

def mwfas2(file_path):
    """
    Main function to find Minimum Weighted Feedback Arc Set (MWFAS) in a graph with parallel edges.

    :param file_path: Path to the file containing the graph.
    :return: A dictionary with metrics, updated graph, removed edges, and their weights.
    """
    # Step 1: Read edges from file
    edges = read_graph(file_path)
    n = max(max(u, v) for u, v, _, _ in edges) + 1

    # Step 2: Construct DirectedGraph as adjacency list with edge IDs
    graph = defaultdict(list)
    weights = {}
    for u, v, weight, edge_id in edges:
        edge_key = (u, v, edge_id)
        graph[u].append((v, edge_id))
        weights[edge_key] = weight

    print("Graph loaded successfully.")
    print("Weights dictionary:", weights)

    # Original graph statistics
    total_edges = len(edges)
    total_weight = sum(weight for _, _, weight, _ in edges)

    # Phase 1: Compute topological ordering dynamically
    in_degree = {i: 0 for i in range(n)}
    for u in graph:
        for v, _ in graph[u]:
            in_degree[v] += 1

    queue = deque([node for node in range(n) if in_degree[node] == 0])
    ordering = []
    while queue:
        node = queue.popleft()
        ordering.append(node)
        for neighbor, _ in graph[node]:
            in_degree[neighbor] -= 1
            if in_degree[neighbor] == 0:
                queue.append(neighbor)

    # Ensure all nodes are in the ordering
    if len(ordering) < n:
        print("Warning: Graph contains cycles or disconnected components.")
        for i in range(n):
            if i not in ordering:
                ordering.append(i)

    # Phase 2: Compute removed edges (those not respecting the ordering)
    # Phase 2: Compute removed edges (those not respecting the ordering)
    removed_edges = []
    removed_weights = {}
    
    for u in graph:
        for v, edge_id in graph[u]:
            edge_key = (u, v, edge_id)
            if ordering.index(u) > ordering.index(v):
                removed_edges.append(edge_key)
                if edge_key in weights:
                    removed_weights[edge_key] = weights[edge_key]
                else:
                    print(f"Warning: Weight missing for edge {edge_key}. Defaulting to 0.")
                    removed_weights[edge_key] = 0
    
    # Remove invalid edges from graph
    for u, v, edge_id in removed_edges:
        graph[u] = [(neighbor, eid) for neighbor, eid in graph[u] if not (neighbor == v and eid == edge_id)]
    
    # Synchronize weights with the modified graph
    updated_weights = {
        edge_key: weights[edge_key]
        for u in graph
        for v, edge_id in graph[u]
        if (edge_key := (u, v, edge_id)) in weights
    }
    
    weights = updated_weights
    num_removed_edges = len(removed_edges)  # Number of edges removed
    total_removed_weight = sum(removed_weights.values())  # Total weight of removed edges

# Return results
    return {
    "total_edges": total_edges,
    "total_weight": total_weight,
    "num_removed_edges": num_removed_edges,
    "removed_weight": total_removed_weight,
    "final_graph": graph,
    "removed_edges": removed_edges,
    "removed_weights": removed_weights,
    }


In [3]:
def compute_vertex_rankings(graph, weights, n):
    """
    Compute rankings for the vertices in a DAG with parallel edges.
    :param graph: Adjacency list of the DAG with (neighbor, edge_id) pairs.
    :param weights: Dictionary of edge weights with keys (u, v, edge_id).
    :param n: Total number of vertices in the graph.
    :return: A list of rankings for the vertices.
    """
    # Step 1: Calculate in-degrees
    in_degree = [0] * n
    for u in graph:
        for v, _ in graph[u]:  # Ignore edge_id for in-degree calculation
            in_degree[v] += 1

    # Step 2: Perform topological sort using a min-heap
    from heapq import heappop, heappush
    min_heap = []
    for i in range(n):
        if in_degree[i] == 0:
            heappush(min_heap, i)

    topological_order = []
    while min_heap:
        current = heappop(min_heap)
        topological_order.append(current)
        for neighbor, _ in graph[current]:  # Ignore edge_id for topological sort
            in_degree[neighbor] -= 1
            if in_degree[neighbor] == 0:
                heappush(min_heap, neighbor)

    # Step 3: Calculate outgoing and incoming edge weight sums for all vertices
    outgoing_weights = {v: 0 for v in range(n)}
    incoming_weights = {v: 0 for v in range(n)}

    for u in graph:
        for v, edge_id in graph[u]:
            edge_key = (u, v, edge_id)
            outgoing_weights[u] += weights.get(edge_key, 0)
            incoming_weights[v] += weights.get(edge_key, 0)

    # Step 4: Assign rankings
    rankings = [-1] * n
    current_rank = 0
    for vertex in topological_order:
        rankings[vertex] = current_rank
        current_rank += 1

    # Break ties for vertices with the same ranking based on outgoing and incoming edge weights
    tied_vertices = sorted(
        [(rankings[v], -(outgoing_weights[v] - incoming_weights[v]) / 
          (outgoing_weights[v] + incoming_weights[v] if outgoing_weights[v] + incoming_weights[v] > 0 else 1), v)
         for v in range(n)],
        key=lambda x: (x[0], x[1])  # Sort by rank first, then by normalized weight difference
    )

    scores = [0] * n
    for final_rank, (_, _, vertex) in enumerate(tied_vertices):
        scores[vertex] = n - final_rank - 1

    return scores


In [4]:
# def optimize_updated_scores(adjacency_matrix, scores, epsilon=1e-8):
#     import cvxpy as cp
#     import numpy as np

#     n = len(scores)

#     # Convert inputs to numpy
#     adjacency_matrix_np = adjacency_matrix.numpy()
#     scores_np = scores.numpy().flatten()

#     # Define optimization variables
#     updated_scores = cp.Variable(n)
#     R = cp.Variable((n, n))  # Auxiliary variable for ratios

#     # Compute M (skew-symmetric pairwise matrix)
#     M = adjacency_matrix_np - adjacency_matrix_np.T

#     # Edge mask
#     edge_mask = (adjacency_matrix_np + adjacency_matrix_np.T > 0).astype(float)

#     # Objective: Minimize squared difference between M and R
#     objective = cp.sum_squares(cp.multiply(edge_mask, M - R))

#     # Constraints for R_ij
#     constraints = []
#     for i in range(n):
#         for j in range(n):
#             if edge_mask[i, j] > 0:
#                 constraints.append(R[i, j] * (updated_scores[i] + updated_scores[j] + epsilon) == updated_scores[i] - updated_scores[j])

#     # Order-preserving constraints
#     constraints += [
#         updated_scores[i] <= updated_scores[j]
#         for i in range(n)
#         for j in range(n)
#         if scores_np[i] <= scores_np[j]
#     ]

#     # Solve the problem
#     problem = cp.Problem(cp.Minimize(objective), constraints)
#     problem.solve()

#     return updated_scores.value


In [5]:
import torch

def graph_to_adjacency_matrix(graph, weights, n):
    """
    Convert the graph to an adjacency matrix with weights.
    :param graph: Adjacency list of the DAG with (neighbor, edge_id) pairs.
    :param weights: Dictionary of edge weights with keys (u, v, edge_id).
    :param n: Total number of vertices in the graph.
    :return: An adjacency matrix (n x n) with weights as a PyTorch tensor.
    """
    import torch
    adjacency_matrix = torch.zeros((n, n), dtype=torch.float32)

    for u in graph:
        for v, edge_id in graph[u]:
            edge_key = (u, v, edge_id)
            adjacency_matrix[u, v] = weights.get(edge_key, 0)

    return adjacency_matrix








In [6]:
import torch
import numpy as np

def reorder_floats(x):
    n = len(x)
    random_floats = np.random.uniform(0,  2*n/3, n)
    y = np.zeros(n)
    for idx, val in enumerate(np.argsort(x)):
        y[val] = sorted(random_floats)[idx]
    return y

def calculate_upset_loss(adjacency_matrix, scores, style='ratio', margin=0.01):
    """
    Calculate the upset loss for the graph rankings using adjacency matrix and scores.

    :param adjacency_matrix: Torch FloatTensor adjacency matrix (n x n).
    :param scores: Torch FloatTensor ranking scores (n x 1).
    :param style: Type of upset loss ('naive', 'simple', 'ratio', or 'margin').
    :param margin: Margin for margin loss (default: 0.01).
    :return: Torch FloatTensor upset loss value.
    """
    epsilon = 1e-8  # For numerical stability

    # Ensure scores are 2D
    if scores.ndim == 1:
        scores = scores.view(-1, 1)

    # Skew-symmetric pairwise comparison matrix (M)
    M1 = adjacency_matrix - adjacency_matrix.T

    # Normalize scores to [0, 1] range
    normalized_scores = scores

    # Pairwise score differences (T)
    T1 = normalized_scores - normalized_scores.T

    # Edge mask: Only consider meaningful edges (where M != 0)
    edge_mask = M1 != 0

    if style == 'ratio':
        min_upset = float('inf')  # Initialize with a large value
        
        for _ in range(40):
            # Generate reordered scores using reorder_floats
            if _==0:
                reordered_scores=scores
            else:
                reordered_scores = torch.FloatTensor(reorder_floats(scores.flatten().tolist()))
            reordered_scores = reordered_scores.view(-1, 1)

            # Compute T2 for normalized scores
            T2 = reordered_scores + reordered_scores.T + epsilon
            T = torch.div(T1, T2)
            M2 = adjacency_matrix + adjacency_matrix.T + epsilon
            M3 = torch.div(M1, M2)  # Normalize the adjacency matrix
            
            # Compute ratio-based upset loss for this iteration
            powers = torch.pow((M3 - T)[edge_mask], 2)
            upset_loss = torch.sum(powers) / torch.sum(edge_mask)

            # Track the minimum upset loss
            min_upset = min(min_upset, upset_loss.item())
        
        return torch.tensor(min_upset)

    elif style == 'naive':
        upset = torch.sum(torch.sign(T1[edge_mask]) != torch.sign(M1[edge_mask])) / torch.sum(edge_mask)

    elif style == 'simple':
        upset = torch.mean((torch.sign(T1[edge_mask]) - torch.sign(M1[edge_mask]))**2)

    elif style == 'margin':
        upset = torch.mean(torch.nn.functional.relu(-M1[edge_mask] * (T1[edge_mask] - margin)))

    else:
        raise ValueError(f"Unsupported style: {style}")

    return upset


In [7]:
import torch

def compute_ratio_upset_loss(adjacency_matrix, scores, epsilon=1e-8):
    """
    Compute the ratio upset loss for the graph rankings using adjacency matrix and scores.

    :param adjacency_matrix: Torch FloatTensor adjacency matrix (n x n).
    :param scores: Torch FloatTensor ranking scores (n x 1).
    :param epsilon: Small value for numerical stability (default: 1e-8).
    :return: Torch FloatTensor ratio upset loss value.
    """
    # Ensure scores are 2D
    if scores.ndim == 1:
        scores = scores.view(-1, 1)

    # Skew-symmetric pairwise comparison matrix (M)
    M1 = adjacency_matrix - adjacency_matrix.T

    # Pairwise score differences (T1)
    T1 = scores - scores.T

    # Edge mask: Only consider meaningful edges (where M1 != 0)
    edge_mask = M1 != 0

    # Compute T2 for normalized scores
    T2 = scores + scores.T + epsilon
    T = torch.div(T1, T2)

    # Normalize M1 using adjacency matrix
    M2 = adjacency_matrix + adjacency_matrix.T + epsilon
    M3 = torch.div(M1, M2)  # Normalize the adjacency matrix

    # Compute ratio upset loss
    powers = torch.pow((M3 - T)[edge_mask], 2)
    upset_loss = torch.sum(powers) / torch.sum(edge_mask)

    return upset_loss


In [8]:
def minimize_ratio_upset_loss(adjacency_matrix, scores, epsilon=1e-2, max_time=120):
    """
    Perform optimization to minimize the ratio upset loss, ensuring naive and simple losses do not worsen.
    :param adjacency_matrix: Torch FloatTensor adjacency matrix (n x n).
    :param scores: Initial scores for optimization.
    :param epsilon: Small value for numerical stability (default: 1e-2).
    :param max_time: Maximum time for optimization in seconds.
    :return: Tuple (optimal_scores, minimized_loss)
    """
    import numpy as np
    import torch
    from scipy.optimize import minimize
    import time

    n = adjacency_matrix.shape[0]

    # Compute initial losses
    initial_scores = scores.clone().detach().view(-1).numpy()
    initial_losses = {
        "naive": calculate_upset_loss(adjacency_matrix, scores.view(-1, 1), style="naive"),
        "simple": calculate_upset_loss(adjacency_matrix, scores.view(-1, 1), style="simple"),
    }

    # Objective function
    def objective_function(updated_scores, adjacency_matrix, initial_losses):
        updated_scores = torch.tensor(updated_scores, dtype=torch.float32).view(-1, 1)
        ratio_loss = calculate_upset_loss(adjacency_matrix, updated_scores, style="ratio")
        naive_loss = calculate_upset_loss(adjacency_matrix, updated_scores, style="naive")
        simple_loss = calculate_upset_loss(adjacency_matrix, updated_scores, style="simple")

        penalty = 0
        if naive_loss > initial_losses["naive"]:
            penalty += naive_loss - initial_losses["naive"]
        if simple_loss > initial_losses["simple"]:
            penalty += simple_loss - initial_losses["simple"]

        return ratio_loss + 100 * penalty

    # Timer callback
    class TimerCallback:
        def __init__(self, max_time, objective_function, adjacency_matrix, initial_losses):
            self.start_time = time.time()
            self.max_time = max_time
            self.iterations = 0  # Track iterations
            self.objective_function = objective_function
            self.adjacency_matrix = adjacency_matrix
            self.initial_losses = initial_losses
            self.min_loss = float("inf")  # Track minimum loss

        def __call__(self, xk, *args, **kwargs):
            self.iterations += 1

            # Compute the objective function value
            current_loss = self.objective_function(
                xk, self.adjacency_matrix, self.initial_losses
            )

            # Update and print the minimum loss found so far
            self.min_loss = min(self.min_loss, current_loss)
            print(f"Iteration {self.iterations}: Minimum loss so far: {self.min_loss:.6f}")

            # Stop optimization if the time limit is exceeded
            if time.time() - self.start_time > self.max_time:
                print("Time limit exceeded, stopping optimization.")
                raise StopIteration  # Signal COBYLA to stop

    # Create the callback instance
    callback = TimerCallback(
        max_time=max_time,
        objective_function=objective_function,
        adjacency_matrix=adjacency_matrix,
        initial_losses=initial_losses,
    )

    try:
        # Run optimization with COBYLA
        result = minimize(
            fun=objective_function,
            x0=initial_scores,
            method="COBYLA",
            args=(adjacency_matrix, initial_losses),  # Pass required arguments
            options={"maxiter": 500, "disp": False},
            callback=callback,  # Logs the minimum loss after each iteration
        )
    except StopIteration:
        print("Optimization stopped early due to time limit.")

    # Extract results
    optimal_scores = result.x
    minimized_loss = result.fun

    print(f"Optimization completed after {callback.iterations} iterations.")
    print(f"Final minimum loss: {callback.min_loss:.6f}")
    return optimal_scores, minimized_loss





In [9]:
def evaluate_upset_losses(file_path, rankings):
    """
    Evaluate upset losses (naive, simple, ratio, margin) for a graph and given rankings.

    :param file_path: Path to the graph file.
    :param rankings: List of rankings for the vertices.
    """
    # Step 1: Prepare Graph and Adjacency Matrix
    edges = read_graph(file_path)
    n = max(max(u, v) for u, v, _ in edges) + 1
    graph, weights = initialize_graph(edges)
    adjacency_matrix = graph_to_adjacency_matrix(graph, weights, n)

    # Step 2: Convert Rankings to Scores Tensor
    scores = torch.FloatTensor(rankings).view(-1, 1)

    # Step 3: Calculate Upset Losses
    naive_loss = calculate_upset_loss(adjacency_matrix, scores, style='naive').item()
    simple_loss = calculate_upset_loss(adjacency_matrix, scores, style='simple').item()
    ratio_loss = calculate_upset_loss(adjacency_matrix, scores, style='ratio').item()
    margin_loss = calculate_upset_loss(adjacency_matrix, scores, style='margin').item()

    # Step 4: Print Results
    print("Upset Losses for the Graph Rankings:")
    print(f"Naive Upset Loss: {naive_loss:.4f}")
    print(f"Simple Upset Loss: {simple_loss:.4f}")
    print(f"Differentiable Upset Loss (Ratio): {ratio_loss:.4f}")
    print(f"Upset Margin Loss: {margin_loss:.4f}")


In [12]:
# Main Execution
import time
import pandas as pd
import torch

# List of input files
file_paths = ["1987adj.txt"]

# Initialize results storage
results = []

# Iterate through each input file
for file_path in file_paths:
    start_time = time.time()

    # Step 1: Read the graph and initialize structures
    edges = read_graph(file_path)
    n = max(max(u, v) for u, v, _, _ in edges) + 1
    print(f"Processing file: {file_path} - Number of nodes: {n}, Number of edges: {len(edges)}")

    init_graph, init_weights = initialize_graph(edges)

    # Step 2: Ensure the graph is a DAG by removing cycles
    result = mwfas(file_path)
    new_graph = result['final_graph']
    new_weights = {key: value for key, value in init_weights.items() if key not in result['removed_weights']}

    # Step 3: Compute rankings for the vertices using the modified graph
    final_rankings = compute_vertex_rankings(new_graph, new_weights, n)

    # Step 4: Evaluate upset losses before optimization
    scores = torch.FloatTensor(final_rankings).view(-1, 1)
    adjacency_matrix = graph_to_adjacency_matrix(init_graph, init_weights, n)

    print("Evaluating losses before optimization...")
    naive_loss_before = calculate_upset_loss(adjacency_matrix, scores, style='naive')
    simple_loss_before = calculate_upset_loss(adjacency_matrix, scores, style='simple')
    ratio_loss_before = calculate_upset_loss(adjacency_matrix, scores, style='ratio')
    end_time = time.time()
    elapsed_time = end_time - start_time
    print("Before Optimization:")
    print(f"Naive Loss: {naive_loss_before}")
    print(f"Simple Loss: {simple_loss_before}")
    print(f"Ratio Loss: {ratio_loss_before}")
    print(f"Elapsed Time: {elapsed_time:.4f} seconds")
    # Step 5: Perform optimization
  #  optimized_scores = minimize_ratio_upset_loss(adjacency_matrix, scores)[0]
  #  optimized_scores = torch.FloatTensor(optimized_scores).view(-1, 1)  # Ensure Torch Tensor format

    # Step 6: Evaluate upset losses after optimization
   # print("Evaluating losses after optimization...")
   # naive_loss_after = calculate_upset_loss(adjacency_matrix, optimized_scores, style='naive')
   # simple_loss_after = calculate_upset_loss(adjacency_matrix, optimized_scores, style='simple')
   # ratio_loss_after = calculate_upset_loss(adjacency_matrix, optimized_scores, style='ratio')

   # end_time = time.time()
   # elapsed_time = end_time - start_time

    # Step 7: Print final results
   # print("After Optimization:")
   # print(f"Naive Loss: {naive_loss_after}")
   # print(f"Simple Loss: {simple_loss_after}")
   # print(f"Ratio Loss: {ratio_loss_after}")
   # print(f"Elapsed Time: {elapsed_time:.4f} seconds")

   
    
    # Store results in a list
    # results.append({
    #     "File": "Basketball_"+str(file_path[:9]),
    #     "Nodes": n,
    #     "Edges": len(edges),
    #     "Naive Loss": round(naive_loss.item(),2),
    #     "Simple Loss": round(simple_loss.item(),2),
    #     "Ratio Loss": round(ratio_loss.item(),2),
    #     "Elapsed Time (s)": round(elapsed_time,2)
    # })
    # print
    # print("simple loss=",simple_loss.item())

# Write results to an Excel file
#output_file = "graph_analysis_results.xlsx"
#df = pd.DataFrame(results)
#df.to_excel(output_file, index=False)

#print(f"Results written to {output_file}")




Processing file: 1987adj.txt - Number of nodes: 290, Number of edges: 3045
Evaluating losses before optimization...
Before Optimization:
Naive Loss: 0.13031385838985443
Simple Loss: 0.5212554335594177
Ratio Loss: 0.5652036070823669
Elapsed Time: 2.0653 seconds


In [11]:
file_paths = ["England 2009 2010.txt"]
results = []

for file_path in file_paths:
    start_time = time.time()

    # Step 1: Read and process the graph
    edges = read_graph(file_path)
    n = max(max(u, v) for u, v, _, _ in edges) + 1  # Get number of nodes
    print(f"Processing file: {file_path} - Number of nodes: {n}, Number of edges: {len(edges)}")

    # Step 2: Ensure the graph is a DAG by removing cycles
    result = mwfas2(file_path)
    new_graph = result['final_graph']
    removed_weights = result['removed_weights']
    total_edges = result['total_edges']
    total_removed_weight = result['removed_weight']

    for u in new_graph:
        for v, edge_id in new_graph[u]:
            edge_key = (u, v, edge_id)
            if edge_key not in removed_weights:
                print(f"Warning: Missing weight for edge {edge_key}. Defaulting to 0.")
                removed_weights[edge_key] = 0

    print(f"Graph processed: Total edges: {total_edges}, Removed weight: {total_removed_weight}")

    # Step 3: Compute rankings for the vertices using the modified graph
    final_rankings = compute_vertex_rankings(new_graph, removed_weights, n)
    print("Final graph:", new_graph)
    print("Removed weights:", removed_weights)

    # Step 4: Evaluate upset losses before optimization
    scores = torch.FloatTensor(final_rankings).view(-1, 1)
    adjacency_matrix = graph_to_adjacency_matrix(new_graph, removed_weights, n)

    print("Evaluating losses before optimization...")
    naive_loss_before = calculate_upset_loss(adjacency_matrix, scores, style='naive')
    simple_loss_before = calculate_upset_loss(adjacency_matrix, scores, style='simple')
    ratio_loss_before = calculate_upset_loss(adjacency_matrix, scores, style='ratio')

    end_time = time.time()
    elapsed_time = end_time - start_time
    print("Before Optimization:")
    print(f"Naive Loss: {naive_loss_before}")
    print(f"Simple Loss: {simple_loss_before}")
    print(f"Ratio Loss: {ratio_loss_before}")
    print(f"Elapsed Time: {elapsed_time:.4f} seconds")


Processing file: England 2009 2010.txt - Number of nodes: 20, Number of edges: 164
Graph loaded successfully.
Weights dictionary: {(10, 14, 0): 1.0, (12, 13, 1): 5.0, (17, 9, 2): 5.0, (2, 8, 3): 1.0, (11, 4, 4): 5.0, (15, 3, 5): 2.0, (0, 7, 6): 2.0, (18, 6, 7): 1.0, (1, 16, 8): 1.0, (2, 17, 9): 7.0, (10, 19, 10): 1.0, (6, 3, 11): 2.0, (18, 7, 12): 2.0, (16, 8, 13): 4.0, (1, 0, 14): 4.0, (11, 3, 15): 6.0, (6, 0, 16): 1.0, (8, 9, 17): 1.0, (12, 19, 18): 4.0, (17, 13, 19): 1.0, (18, 10, 20): 10.0, (4, 7, 21): 1.0, (2, 15, 22): 3.0, (16, 5, 23): 3.0, (14, 1, 24): 1.0, (1, 9, 25): 3.0, (2, 7, 26): 4.0, (18, 11, 27): 3.0, (0, 17, 28): 1.0, (16, 6, 29): 1.0, (14, 15, 30): 4.0, (4, 10, 31): 2.0, (12, 3, 32): 3.0, (13, 19, 33): 2.0, (1, 7, 34): 8.0, (12, 11, 35): 2.0, (9, 3, 36): 1.0, (2, 0, 37): 8.0, (17, 8, 38): 4.0, (18, 16, 39): 4.0, (14, 6, 40): 2.0, (11, 10, 41): 3.0, (14, 3, 42): 3.0, (9, 0, 43): 1.0, (7, 17, 44): 1.0, (6, 8, 45): 1.0, (1, 5, 46): 4.0, (2, 16, 47): 2.0, (4, 13, 48): 4.0,